<a href="https://colab.research.google.com/github/lookinsight/ml/blob/main/20221110_ML_DecisionTree_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 결정트리


In [ ]:
# https://www.kaggle.com/datasets/ayessa/salary-prediction-classification
# https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/salary.csv
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

## 데이터 불러오기

In [ ]:
file_url = 'https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/salary.csv'
df = pd.read_csv(file_url, skipinitialspace = True) 

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
# [연속형 변수]
# age: 연령
# education-num : 교육년수
# capital-gain : 자산증가
# capital-loss : 자삼감소
# hours-per-week : 주당 노동 시간
df.describe()

In [ ]:
# [범주형 변수]
# workclass : 고용형태
# education : 학력
# marital-status : 결혼 상태
# occupation : 직업
# relationship : 가족관계
# race : 인종
# sex : 성별
# native-country: 출신국가
# class: 연봉 범위

df.describe(include = ['O'])

In [ ]:
 df.describe(include = 'all') 

In [ ]:
df['capital-gain'].plot()	

## 전처리

1. 전처리 할 때
2. 전처리 -> **범주형 변수** (drop? 연속형 변수처리? dummies? 대체?) / **결측치** / (스케일링 -> 아웃라이어는 신경 안씀. 왜 결정트리라서)

2-1. education : education-num -> drop<br> 
2-2. occupation : 이미 묵여 있고 중요할 듯 -> dummies<br> 
2-3. native-country : 출신국가들인데 어차피 결정트리니까 고연봉자 비율 정도로만 처리해줘도 괜찮을듯?

In [ ]:
df['class']

In [ ]:
df['class'].value_counts()

In [ ]:
df['class'] = df['class'].map({'<=50K': 0, '>50K': 1})
df['class']

In [ ]:
df['class'].value_counts()

In [ ]:
df['age'].dtype

In [ ]:
for c in df.columns:
    print(c, df[c].dtype)

In [ ]:
obj_list = []
for c in df.columns:
    if df[c].dtype == 'object':
        obj_list.append(c)
        # print(c, df[c].dtype)
print(obj_list)

obj_list2 = [c for c in df.columns if df[c].dtype == 'object']
print(obj_list2) 

In [ ]:
# 1. 연속형 변수? <- 1, 2, 3... Group의 평균, 최빈값 
# 2. drop? 
# 3. dummies? 
for o in obj_list:
    if df[o].nunique() > 10:
        print(o,'-', df[o].nunique()) 

In [ ]:
df.education.value_counts()

In [ ]:
df['education-num']

In [ ]:
for n in range(1, 17):
    # print(f"**{n}**") 
    # print(df[df['education-num'] == n]['education'])
    print(f"**{n}**", df[df['education-num'] == n]['education'].unique())
    # print(df[df['education-num'] == n]['education'].nunique())
    # print(df[df['education-num'] == n]['education'].nunique() == 1)

In [ ]:
df.drop(columns = ['education'], axis = 1, inplace = True)

In [ ]:
df.info()

In [ ]:
# 데이터 이미 묶여있고, 직업이라는 데이터 특성상 연봉 큰 영향 예상 
df['occupation'].value_counts()

In [ ]:
df['native-country'].value_counts()  # 미국이 가장 많다 나머지?
# 1. 미국 vs 기타 : 0, 1
# 2. 대륙별, 언어권별 -> 추가적인 기준을 만들어 더미변수화
# 3. 

In [ ]:
# groupby(묶고 싶은 열 이름)[특정한 열 이름], 적용하고자 하는 그룹함수() => mean()
# df.groupby(['native-country']).min()
# df.groupby(['native-country']).mean()
# df.groupby(['native-country'])['class'].mean()
df.groupby(['native-country'])['class'].mean().sort_values(ascending = False) 

In [ ]:
# df[df['native-country'] == 'France'].groupby(['occupation'])
df[df['native-country'] == 'France'].groupby(['occupation'])['class'].mean() 

In [ ]:
country_group = df.groupby('native-country').mean()['class']
country_group 
# country_group 나라별 고연봉자 비율을 원래 df 에 합치고 싶음
# 나라별 -> 나라 이름 -> index에 있음 -> reset_index 인덱스에서 꺼내기

In [ ]:
country_group.index

In [ ]:
country_group = country_group.reset_index()
country_group

In [ ]:
# df.groupby(['native-country']).mean()[['class_x','age']].sort_values(by = 'age', ascending = False)['class_x']

In [ ]:
# A.merge(B, on = [기준이 되는것]) A 왼쪽, B 오른쪽 
# A 라고 하는 쪽 결측치 있을 수 있음
# A를 모두 보존해주는 방향으로 -> B가 없어도 돼(country_group.class) -> how = 'left'
# index가 기본인데,, 아니면 on으로 열(컬럼) 지정
# native-country? : 결측치... 
df = df.merge(country_group, on = 'native-country', how = 'left')
df

In [ ]:
# df['country_class_mean']
# class x, class y 
df.drop('native-country', axis=1, inplace=True)
df

In [ ]:
df = df.rename(columns = {'class_x': 'class', 'class_y':'native-country'})
df

# 결측치 처리 & 더미 변수 변환

In [ ]:
df.isna().mean()

## 임의값 넣어주기

In [ ]:
df['native-country'].fillna(-99, inplace=True)
df.isna().mean()

In [ ]:
df['workclass'].value_counts() / len(df)

In [ ]:
df['workclass'].fillna('Private', inplace=True)
df.isna().mean()

In [ ]:
df['occupation'].value_counts()

In [ ]:
df['occupation'].fillna('Unknown', inplace=True)
df.isna().mean()

In [ ]:
df.info()

In [ ]:
df2 = pd.get_dummies(df, drop_first=True)
df2.info()

# 모델링 평가

## 훈련셋 & 시험셋 분리

In [ ]:
#@title 훈련셋 & 시험셋 분리
from sklearn.model_selection import train_test_split
X = df2.drop('class', axis=1)
y = df2['class']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=100
)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(random_state = 100)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)